In [1]:
file_path = "datasets/semeval16/restaurant_2016_testing_gold_coba_coba.xml.seg"

with open(file_path, "r") as file:
    content = file.read()

print(content)


Serves really good $T$ .
sushi
1
Not the biggest $T$ but adequate .
portions
0
$T$ is a must !
Green Tea creme brulee
1
â€“ I can't say enough about this $T$ .
place
1
It has great $T$ and even better service .
sushi
1
It has great sushi and even better $T$ .
service
1
The entire $T$ was extremely accomodating and tended to my every need .
staff
1
I've been to this $T$ over a dozen times with no complaints to date .
restaurant
1
The $T$ is belligerent to guests that have a complaint .
owner
-1
Good $T$ !
food
1
This is a great place to get a delicious $T$ .
meal
1
The $T$ is pretty friendly .
staff
1
The $T$ are great !
onion rings
1
â€“ I was highly disappointed in the $T$ at Pagoda .
food
-1
The $T$ tasted like sticky sweet donuts and the honey walnut prawns, the few they actually give you.....were not good .
lemon chicken
-1
The lemon chicken tasted like sticky sweet donuts and the $T$, the few they actually give you.....were not good .
honey walnut prawns
-1
The lemon chicken taste

In [2]:
# Split the content into lines
lines = content.split('\n')

# Create three lists starting from different lines
list1 = lines[::3]
list2 = lines[1::3]
list3 = lines[2::3]
list1.pop()
# Print the lists
print("List 1:", list1)
print("List 2:", list2)
print("List 3:", list3)

print("Length of List 1:", len(list1))
print("Length of List 2:", len(list2))
print("Length of List 3:", len(list3))


List 1: ['Serves really good $T$ .', 'Not the biggest $T$ but adequate .', '$T$ is a must !', "â€“ I can't say enough about this $T$ .", 'It has great $T$ and even better service .', 'It has great sushi and even better $T$ .', 'The entire $T$ was extremely accomodating and tended to my every need .', "I've been to this $T$ over a dozen times with no complaints to date .", 'The $T$ is belligerent to guests that have a complaint .', 'Good $T$ !', 'This is a great place to get a delicious $T$ .', 'The $T$ is pretty friendly .', 'The $T$ are great !', 'â€“ I was highly disappointed in the $T$ at Pagoda .', 'The $T$ tasted like sticky sweet donuts and the honey walnut prawns, the few they actually give you.....were not good .', 'The lemon chicken tasted like sticky sweet donuts and the $T$, the few they actually give you.....were not good .', 'The lemon chicken tasted like sticky sweet donuts and the $T$, the few they actually give you.....were not good .', 'Nice $T$, but highly overrated p

In [3]:
# import string

# for val1, val2, val3 in zip(list1, list2, list3):
#     val1 = val1.strip()
#     if val1[-1] in string.punctuation and val1[-1] != "$":
#         val1 = val1[:-1] + " " + val1[-1]
#     print("List 1:", val1)
#     print("List 2:", val2)
#     print("List 3:", val3)


In [4]:
modified_list = []
list_ada_isinya = []
for i in range(len(list1)):
    list_ada_isinya.append(list1[i].replace('$T$', list2[i]))
    modified_list.append(list1[i].replace('$T$', ''))
print(modified_list)
print(list_ada_isinya)

['Serves really good  .', 'Not the biggest  but adequate .', ' is a must !', "â€“ I can't say enough about this  .", 'It has great  and even better service .', 'It has great sushi and even better  .', 'The entire  was extremely accomodating and tended to my every need .', "I've been to this  over a dozen times with no complaints to date .", 'The  is belligerent to guests that have a complaint .', 'Good  !', 'This is a great place to get a delicious  .', 'The  is pretty friendly .', 'The  are great !', 'â€“ I was highly disappointed in the  at Pagoda .', 'The  tasted like sticky sweet donuts and the honey walnut prawns, the few they actually give you.....were not good .', 'The lemon chicken tasted like sticky sweet donuts and the , the few they actually give you.....were not good .', 'The lemon chicken tasted like sticky sweet donuts and the , the few they actually give you.....were not good .', 'Nice , but highly overrated place .', 'Nice ambience, but highly overrated  .', 'Worst  I E

In [5]:
# -*- coding: utf-8 -*-

import numpy as np
import networkx as nx
import spacy

from spacy.tokens import Doc

class WhitespaceTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        words = text.split()
        # All tokens 'own' a subsequent space character in this tokenizer
        spaces = [True] * len(words)
        return Doc(self.vocab, words=words, spaces=spaces)

nlp = spacy.load('en_core_web_sm')
nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)

def dependency_dist_func(text, aspect_term):

    # https://spacy.io/docs/usage/processing-text
    document = nlp(text)
    # Load spacy's dependency tree into a networkx graph
    edges = []
    
    list2 = []
    words = []
    for token in document:
        # FYI https://spacy.io/docs/api/token
        # print("token:", token.dep_)
        list2.append(token.dep_)
        words.append(token.text)
        for child in token.children:
            # Comprehensive print please
            # print("token: ", token.i, token.text, token.dep_, token.head.i, token.head.text, [child for child in token.children])
            edges.append((token.i, child.i))
    # print("edges: ", edges)
    graph = nx.Graph(edges)

    text_lst = text.split()
    seq_len = len(text_lst)
    text_left, _, _ = text.partition(aspect_term)
    start = len(text_left.split())
    end = start + len(aspect_term.split())
    asp_idx = [i for i in range(start, end)]
    dist_matrix = seq_len*np.ones((seq_len, len(asp_idx))).astype('float32')
    for i, asp in enumerate(asp_idx):
        for j in range(seq_len):
            try:
                dist_matrix[j][i] = nx.shortest_path_length(graph, source=asp, target=j)
            except:
                dist_matrix[j][i] = seq_len/2
    dist_matrix = np.min(dist_matrix, axis=1)
    
    for i in range(len(dist_matrix)):
        # print("list2[i]:", list2[i])
        if list2[i] == "neg" or list2[i] == "advmod" or list2[i] == "amod" or list2[i] == "attr" \
            or list2[i] == "ccomp" or list2[i] == "acomp":
            dist_matrix[i] = 1
        # if list2[i] == "advmod" or list2[i] == "amod" or list2[i] == "attr" \
        #     or list2[i] == "ccomp" or list2[i] == "acomp":
        #     dist_matrix[i] = 1
    
    aspect_term_split = aspect_term.split()
    for i in range(len(words) - len(aspect_term_split) + 1):
        if words[i:i+len(aspect_term_split)] == aspect_term_split:
            dist_matrix[i:i+len(aspect_term_split)] = 0
            
    return dist_matrix


# $T$ is super fast , around anywhere from 35 seconds to 1 minute .
# Boot time
aspect_term = "food"
text = f"great food but the service was dreadful !"
text = text.replace("$T$", aspect_term)
print(dependency_dist_func(text, aspect_term))


list2[i]: amod
list2[i]: ROOT
list2[i]: cc
list2[i]: det
list2[i]: nsubj
list2[i]: conj
list2[i]: acomp
list2[i]: punct
[1. 0. 1. 3. 2. 1. 1. 2.]


In [98]:
# # -*- coding: utf-8 -*-

# import numpy as np
# import networkx as nx
# import spacy

# from spacy.tokens import Doc

# class WhitespaceTokenizer(object):
#     def __init__(self, vocab):
#         self.vocab = vocab

#     def __call__(self, text):
#         words = text.split()
#         # All tokens 'own' a subsequent space character in this tokenizer
#         spaces = [True] * len(words)
#         return Doc(self.vocab, words=words, spaces=spaces)

# nlp = spacy.load('en_core_web_sm')
# nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)

# def dependency_dist_func(text, aspect_term):
#     # https://spacy.io/docs/usage/processing-text
#     document = nlp(text)
#     # Load spacy's dependency tree into a networkx graph
#     edges = []
    
#     for token in document:
#         # FYI https://spacy.io/docs/api/token
#         for child in token.children:
#             edges.append((token.i, child.i))
#     graph = nx.Graph(edges)

#     text_lst = text.split()
#     seq_len = len(text_lst)
#     text_left, _, _ = text.partition(aspect_term)
#     start = len(text_left.split())
#     end = start + len(aspect_term.split())
#     asp_idx = [i for i in range(start, end)]
#     dist_matrix = seq_len*np.ones((seq_len, len(asp_idx))).astype('float32')
#     for i, asp in enumerate(asp_idx):
#         for j in range(seq_len):
#             try:
#                 dist_matrix[j][i] = nx.shortest_path_length(graph, source=asp, target=j)
#             except:
#                 dist_matrix[j][i] = seq_len/2
#     dist_matrix = np.min(dist_matrix, axis=1)
#     return dist_matrix


In [99]:
list_dep_dist = []
for i in range(len(modified_list)):
    dependency_mat = dependency_dist_func(list_ada_isinya[i], list2[i])
    list_dep_dist.append(dependency_mat)
    # print(dependency_mat)

In [100]:
print(len(list1))

650


In [101]:
for val1, val_dist in zip(list1, list_dep_dist):
    print(val1)
    str_empty = ""
    for val2 in val_dist:
        str_empty += str(val2) + " "
    print(str_empty)

Serves really good $T$ .
1.0 1.0 1.0 0.0 2.0 
Not the biggest $T$ but adequate .
1.0 1.0 1.0 0.0 1.0 1.0 1.0 
$T$ is a must !
0.0 0.0 0.0 0.0 1.0 3.0 1.0 2.0 
â€“ I can't say enough about this $T$ .
5.0 5.0 4.0 3.0 2.0 1.0 1.0 0.0 4.0 
It has great $T$ and even better service .
2.0 1.0 1.0 0.0 1.0 1.0 1.0 1.0 2.0 
It has great sushi and even better $T$ .
3.0 2.0 1.0 1.0 2.0 1.0 1.0 0.0 3.0 
The entire $T$ was extremely accomodating and tended to my every need .
1.0 1.0 0.0 1.0 1.0 1.0 2.0 2.0 3.0 5.0 5.0 4.0 2.0 
I've been to this $T$ over a dozen times with no complaints to date .
3.0 2.0 1.0 1.0 0.0 3.0 6.0 5.0 4.0 3.0 5.0 4.0 5.0 6.0 3.0 
The $T$ is belligerent to guests that have a complaint .
1.0 0.0 1.0 1.0 3.0 4.0 6.0 5.0 7.0 6.0 2.0 
Good $T$ !
1.0 0.0 1.0 
This is a great place to get a delicious $T$ .
4.0 3.0 3.0 1.0 1.0 2.0 1.0 1.0 1.0 0.0 4.0 
The $T$ is pretty friendly .
1.0 0.0 1.0 1.0 1.0 2.0 
The $T$ are great !
1.0 0.0 0.0 1.0 1.0 2.0 
â€“ I was highly disappointed in 

In [102]:
file_path = "datasets/semeval16/restaurant_2016_testing_gold_coba_coba.xml.seg"

with open(file_path, "r") as file:
    content = file.read()

# Read multiple of 3 for each line
lines = content.split('\n')
list1 = lines[::3]
print(len(list1))

651


In [103]:
file_path = "datasets/semeval16/restaurant_2016_testing_gold_coba_coba.xml.seg.dist"

with open(file_path, "r") as file:
    content = file.read()

# Read multiple of 3 for each line
lines = content.split('\n')
list1 = lines[::2]
print(len(list1))

651
